In [1064]:
# autoreload
%load_ext autoreload
%autoreload 2

# fix path
import sys
sys.path.append("/home/jxm/reverse-training")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1075]:
from utils.analysis import load_results_as_df
import os


HOME_DIR = "/home/jxm/reverse-training"
df = load_results_as_df(os.path.join(HOME_DIR, "results", "03-27-heatmap-2"))
df["seed_dataset_size"] = df["select_seed_dataset"].str.extract(r"(\d+)").astype(int)


Found 102 files in /home/jxm/reverse-training/results/03-27-heatmap-2/*.pkl
Filtered 0/102 results with no evaluation metrics
dict_keys(['data', 'time_elapsed', 'full_ot_distance', 'sinkhorn_distance', 'optimal_matching_relaxed_wmd', 'jaccard_overlap_examples', 'jaccard_overlap_vocabulary', 'containment_similarity_examples', 'containment_similarity_vocabulary', 'levenshtein_stats', 'discrete_ot_distance_levenshtein', 'discrete_ot_distance_jaccard', 'args', 'expert_evaluation_metrics', 'evaluation_metrics', 'total_time_in_evaluation'])


In [1076]:
import pandas as pd

pd.set_option('display.max_rows', None)

dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["evaluation_best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_full_ot_distance", "dataset_jaccard_overlap_vocabulary"]

In [1077]:
dfn = df

def fix_name(x):
    if "nq" in x:
        return "NQ"
    elif "ag_news" in x:
        return "AG News"
    elif "rotten_tomatoes" in x:
        return "Rotten Tomatoes"
    elif "imdb" in x:
        return "IMDB"
    elif "dbpedia" in x:
        return "DBpedia"
    elif "newsgroup" in x:
        return "Newsgroup"
    elif "msmarco" in x:
        return "MSMARCO"
    else:
        return "??"

dfn["select_seed_dataset"] = dfn["select_seed_dataset"].apply(fix_name)
dfn["dataset"] = dfn["dataset"].apply(fix_name)


In [1090]:
hdf

                               select_seed_dataset  dataset        
evaluation_best_eval_accuracy  AG News              AG News            0.817708
                                                    DBpedia            0.275065
                                                    IMDB               0.609212
                                                    Newsgroup          0.128092
                                                    Rotten Tomatoes    0.611669
                               DBpedia              AG News            0.504883
                                                    DBpedia            0.603353
                                                    IMDB               0.525553
                                                    Newsgroup          0.081380
                                                    Rotten Tomatoes    0.588800
                               IMDB                 AG News            0.547526
                                                    

In [1092]:
ordering = ["AG News",  "DBpedia",  "IMDB", "Newsgroup", "Rotten Tomatoes", "MSMARCO", "NQ"]
hdf = (
    dfn.groupby(["select_seed_dataset", "dataset"])["evaluation_best_eval_accuracy"]
        .mean()
        .reset_index()
        .pivot(index="dataset", columns="select_seed_dataset", values="evaluation_best_eval_accuracy")
)

# order rows
hdf = hdf[ordering]
hdf = hdf.reindex(ordering)
# order cols
# hdf = hdf.reindex(ordering, axis=1)

import seaborn as sns

# divide each row by the max value of that row
# hdf = hdf.div(hdf.max(axis=1), axis=0)
print(hdf.to_latex(float_format=lambda x: f"{100*x:.1f}"))

\begin{tabular}{lrrrrrrr}
\toprule
select_seed_dataset & AG News & DBpedia & IMDB & Newsgroup & Rotten Tomatoes & MSMARCO & NQ \\
dataset &  &  &  &  &  &  &  \\
\midrule
AG News & 81.8 & 50.5 & 54.8 & 69.9 & 36.3 & 74.4 & 79.2 \\
DBpedia & 27.5 & 60.3 & 17.5 & 19.4 & 11.3 & 48.9 & 59.4 \\
IMDB & 60.9 & 52.6 & 75.6 & 63.7 & 67.5 & 67.1 & 64.8 \\
Newsgroup & 12.8 & 8.1 & 8.2 & 27.8 & 4.6 & 14.1 & 14.2 \\
Rotten Tomatoes & 61.2 & 58.9 & 63.5 & 55.0 & 55.4 & 60.6 & 56.8 \\
MSMARCO & NaN & NaN & NaN & NaN & NaN & NaN & NaN \\
NQ & NaN & NaN & NaN & NaN & NaN & NaN & NaN \\
\bottomrule
\end{tabular}

